## CUDA 内存：

> * local memory：局部内存
* shared memory：共享内存
* constant memory：常量内存
* global memory：全局内存
* unified memory：统一内存
* read-only memory：只读内存
* texture memory：纹理内存

### 1. 寄存器：

寄存器是GPU中最快的内存，一般kernal中没有特殊声明的自动变量都存放在寄存器中。**寄存器是每个线程的私有变量，一旦线程执行结束，寄存器变量就会失效。如果kernal使用的寄存器超过硬件限制，这部分数据会存入局部内存中。**如下图所示：

<img src="image_github/GPU_memory.png" width="400" height="500">

### 2.局部缓存：局部缓存与全局缓存在同一块存储区

### 2.1 什么时候使用局部内存：

>* 编译期间无法确定值的本地数组。
* 消耗太多寄存器的较大结构体或数组。
* 任何超过寄存器限制的变量。

<img src="image_github/local_memory.png" width="400" height="400">

### 3. 共享缓存：



### 4. 常量内存：

**常量内存是用来保存在核函数执行期间不会发生变化的数据。**

### 4.1 常量缓存声明：

<img src="image_github/half_warp.png" width="400" height="400">

**注意：在确定常量内存长度时，不要使用宏定义。因为宏定义在预处理过程读入源代码。**

### 4.2 CPU内存复制到GPU常量内存：

**使用cudaMemcpyToSymbol()将CPU内存数据复制到GPU常量数据中**，其与参数为cudaMemcpyHostToDevice的cudaMemcpy()之间的唯一差异在于，cudaMemcpyToSymbol()会复制到常量内存，而cudaMemcpy()会复制到全局内存。

### 4.3 常量缓存的优点：

* 对常量内存的单次读操作可以广播到其他的“邻近(nearby)”线程，这将节约15次读取操作。

>当处理常量内存时，NVIDIA硬件将把单次内存读取操作广播到每个半线程束(Half-Warp)。在半线程束中包含16个线程，即线程束中线程数量的一半。如果在半线程束中的每个线程从常量内存的相同地址上读取数据，那么GPU只会产生一次读取请求并在随后将数据广播到每个线程。如果从常量内存中读取大量数据，那么这种方式产生的内存流量只是使用全局内存时的1/16。**线程束(warp)：在CUDA架构中，线程束是指一个包含32个线程的集合，这个线程集合被“编织在一起”并且以“步调一致(Lockstep)”的形式执行。在程序中的每一行，线程束中的每个线程都将在不同的数据上执行相同的指令。**

* 常量内存的数据将缓存起来，因此对于相同地址的连续操作将不会产生额外的内存通信量。

>由于这块内存的内容是不发生变化的，因此硬件将主动把这个常量数据缓存在GPU上。在第一次从常量内存的某个地址上读取后，当其他半线程束请求同一个地址时，那么将**命中缓存**（缓存有需要的数据），这同样减少了额外的内存流量。

**注意：虽然当所有16个线程都读取相同地址时，这个功能可以极大提升性能，但当所有16个线程分别读取不同的地址时，它实际上会降低性能。因为这16次不同的读取操作会被串行化，从而需要16倍的时间来发出请求。但如果从全局内存中读取，那么这些请求会同时发出。**

https://blog.csdn.net/Linoi/java/article/details/41522573